# PyTorch Basics Notebook
### Introduction to Tensors, Datasets, DataLoaders, CNNs, and U‑Net Building Blocks

## Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt


## PyTorch Tensors

In [2]:
# Creating tensors
x = torch.tensor([1.0, 2.0, 3.0])
y = torch.randn(3)  # random tensor

print(x)
print(y)

# Tensor operations
print('Addition:', x + y)
print('Mean:', x.mean())


tensor([1., 2., 3.])
tensor([1.2730, 0.1289, 0.1756])
Addition: tensor([2.2730, 2.1289, 3.1756])
Mean: tensor(2.)


## Autograd Basics

In [3]:
# Enable gradient tracking
a = torch.tensor([2.0, 3.0], requires_grad=True)
b = (a * a).sum()
b.backward()
print(a.grad)  # derivative of x^2 is 2x


tensor([4., 6.])


## Custom PyTorch Dataset

We simulate MRI-like slices using random arrays just for practice.

In [4]:
class RandomMRIDataset(Dataset):
    def __init__(self, length=100):
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        image = torch.randn(1, 64, 64)      # fake MRI slice
        mask = (torch.randn(1, 64, 64) > 0).float()  # fake mask
        return image, mask

dataset = RandomMRIDataset()
img, msk = dataset[0]
img.shape, msk.shape


(torch.Size([1, 64, 64]), torch.Size([1, 64, 64]))

## DataLoader

In [5]:
loader = DataLoader(dataset, batch_size=8, shuffle=True)

for images, masks in loader:
    print(images.shape, masks.shape)
    break


torch.Size([8, 1, 64, 64]) torch.Size([8, 1, 64, 64])


## Building a Simple CNN

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        return self.classifier(x)

model = SimpleCNN()
print(model)


SimpleCNN(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=8192, out_features=1, bias=True)
    (2): Sigmoid()
  )
)


## Training Loop Example

In [7]:
model = SimpleCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(2):
    for images, masks in loader:
        optimizer.zero_grad()
        preds = model(images)
        loss = criterion(preds, torch.zeros_like(preds))  # dummy target
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')


Epoch 1, Loss: 0.0000
Epoch 2, Loss: 0.0000


## U‑Net Building Blocks

In [8]:
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU()
    )

sample = torch.randn(1, 1, 128, 128)
block = conv_block(1, 16)
output = block(sample)
output.shape


torch.Size([1, 16, 128, 128])

## Exercises


1. Modify `RandomMRIDataset` to return a resized (128×128) slice using interpolation.  
2. Add another convolution layer to `SimpleCNN` and observe how the model size changes.  
3. Implement a small encoder-decoder network (mini U-Net) using `conv_block`.  
4. Write a custom Dice Loss function in PyTorch.  
5. Train the CNN on the random dataset and plot the loss curve using matplotlib.  


In [18]:
import torch.nn.functional as F

class RandomMRIDataset(Dataset):
    def __init__(self, length=100):
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # Generate initial 64x64 image and resize to 128x128
        image_64 = torch.randn(1, 64, 64)      # fake MRI slice
        image = F.interpolate(image_64.unsqueeze(0), size=(128, 128), mode='bilinear', align_corners=False).squeeze(0)

        # Generate initial 64x64 mask and resize to 128x128
        mask_64 = (torch.randn(1, 64, 64) > 0).float()  # fake mask
        mask = F.interpolate(mask_64.unsqueeze(0), size=(128, 128), mode='bilinear', align_corners=False).squeeze(0)

        return image, mask

dataset = RandomMRIDataset()
img, msk = dataset[0]
print(f"Image shape: {img.shape}, Mask shape: {msk.shape}")

Image shape: torch.Size([1, 128, 128]), Mask shape: torch.Size([1, 128, 128])


In [19]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # Added convolution layer
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        # The input size to the linear layer changes because of the added pooling layer.
        # Original: 64x64 -> MaxPool2d (32x32) -> MaxPool2d (16x16)
        # New: 128x128 (from dataset) -> MaxPool2d (64x64) -> MaxPool2d (32x32) -> MaxPool2d (16x16)
        # Output channels for the last conv layer is 64.
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        return self.classifier(x)

# Instantiate the modified model
model_modified = SimpleCNN()

# Print the model architecture
print("\nModified SimpleCNN Architecture:")
print(model_modified)

# Calculate and print the number of trainable parameters
total_params = sum(p.numel() for p in model_modified.parameters() if p.requires_grad)
print(f"\nTotal trainable parameters: {total_params}")


Modified SimpleCNN Architecture:
SimpleCNN(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16384, out_features=1, bias=True)
    (2): Sigmoid()
  )
)

Total trainable parameters: 39681


In [20]:
class MiniUNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()

        # Encoder path
        self.enc1 = conv_block(in_channels, 16)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = conv_block(16, 32)
        self.pool2 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = conv_block(32, 64)

        # Decoder path
        self.upconv2 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.dec2 = conv_block(32 + 32, 32) # Channels from upconv and skip connection
        self.upconv1 = nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2)
        self.dec1 = conv_block(16 + 16, 16) # Channels from upconv and skip connection

        # Output layer
        self.final_conv = nn.Conv2d(16, out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Encoder
        enc1_out = self.enc1(x)  # 16x128x128
        pool1_out = self.pool1(enc1_out) # 16x64x64

        enc2_out = self.enc2(pool1_out) # 32x64x64
        pool2_out = self.pool2(enc2_out) # 32x32x32

        # Bottleneck
        bottleneck_out = self.bottleneck(pool2_out) # 64x32x32

        # Decoder
        up2_out = self.upconv2(bottleneck_out) # 32x64x64
        # Concatenate with skip connection from enc2_out
        cat2 = torch.cat([up2_out, enc2_out], dim=1) # (32+32)x64x64
        dec2_out = self.dec2(cat2) # 32x64x64

        up1_out = self.upconv1(dec2_out) # 16x128x128
        # Concatenate with skip connection from enc1_out
        cat1 = torch.cat([up1_out, enc1_out], dim=1) # (16+16)x128x128
        dec1_out = self.dec1(cat1) # 16x128x128

        # Final output
        output = self.final_conv(dec1_out)
        return self.sigmoid(output)

# Instantiate the MiniUNet model
mini_unet_model = MiniUNet()
print(mini_unet_model)

# Test with a dummy input (e.g., 1x128x128)
dummy_input = torch.randn(1, 1, 128, 128)
output_shape = mini_unet_model(dummy_input).shape
print(f"Output shape for dummy input: {output_shape}")

MiniUNet(
  (enc1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bottleneck): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (upconv2): ConvTranspose2d(64, 32, kernel_size=(2, 2), stride=(2, 2))
  (dec2): Sequential(
    (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    

In [21]:
def dice_loss(inputs, targets, smooth=1e-6):
    # Flatten label and prediction tensors
    inputs = inputs.view(-1)
    targets = targets.view(-1)

    intersection = (inputs * targets).sum()
    dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

    return 1 - dice

# Test the Dice Loss function with dummy inputs
# Example 1: Perfect overlap
inputs_perfect = torch.tensor([1.0, 1.0, 0.0, 0.0])
targets_perfect = torch.tensor([1.0, 1.0, 0.0, 0.0])
loss_perfect = dice_loss(inputs_perfect, targets_perfect)
print(f"Dice Loss (perfect overlap): {loss_perfect:.4f}")

# Example 2: No overlap
inputs_no_overlap = torch.tensor([1.0, 1.0, 0.0, 0.0])
targets_no_overlap = torch.tensor([0.0, 0.0, 1.0, 1.0])
loss_no_overlap = dice_loss(inputs_no_overlap, targets_no_overlap)
print(f"Dice Loss (no overlap): {loss_no_overlap:.4f}")

# Example 3: Partial overlap
inputs_partial = torch.tensor([1.0, 1.0, 1.0, 0.0])
targets_partial = torch.tensor([0.0, 1.0, 1.0, 1.0])
loss_partial = dice_loss(inputs_partial, targets_partial)
print(f"Dice Loss (partial overlap): {loss_partial:.4f}")


Dice Loss (perfect overlap): 0.0000
Dice Loss (no overlap): 1.0000
Dice Loss (partial overlap): 0.3333
